#   

# 作業目標:

    1. 藉由固定的 dataset, 來驗證不同loss function
    2. Dataset 的特性跟我們選用的loss function 對accrancy 的影響
    
    
# 作業重點: 
    請分別選用 "MSE", "binary _crossentropy"
    查看Train/test accurancy and loss rate
    

# 導入必要的函數

In [1]:
from keras.datasets import cifar10
import numpy as np
np.random.seed(10)

C:\Users\peng\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 資料準備

In [2]:
#取得Keras Dataset
(x_img_train , y_label_train) , (x_img_test , y_label_test) = cifar10.load_data()

In [3]:
#確認 CIFAR10 Dataset 資料維度
print("train data:" ,
      'images:' , x_img_train.shape ,
      " labels:" , y_label_train.shape) 

print("test  data:" , 
      'images:' , x_img_test.shape ,
      " labels:" , y_label_test.shape) 

train data: images: (50000, 32, 32, 3)  labels: (50000, 1)
test  data: images: (10000, 32, 32, 3)  labels: (10000, 1)


In [4]:
#資料正規化
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0

In [5]:
#針對Label 做 ONE HOT ENCODE
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)
y_label_test_OneHot.shape

(10000, 10)

# 訓練模型(loss = 'MSE')

In [6]:
from keras.models import Model
from keras.layers import Input , Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

In [25]:
model_input = Input(shape=(32, 32, 3 , ) , dtype = 'float32' , name = 'main_input')

In [8]:
#卷積層1

In [26]:
conv1 = Conv2D(filters = 32 , 
               kernel_size = (3 , 3) ,
               input_shape = (32 , 32 , 3), 
               activation = 'relu', 
               padding = 'same')(model_input)

conv1 = Dropout(rate = 0.25)(conv1)

pooled1 = MaxPooling2D(pool_size = (2 , 2))(conv1)

In [12]:
#卷積層2與池化層2

In [27]:
conv2 = Conv2D(filters = 64 , 
               kernel_size = (3 , 3) ,
               input_shape = (32 , 32 , 3), 
               activation = 'relu', 
               padding = 'same')(pooled1)

conv2 = Dropout(rate = 0.25)(conv2)

pooled2 = MaxPooling2D(pool_size = (2 , 2))(conv2)

In [16]:
#建立神經網路(平坦層、隱藏層、輸出層)

In [28]:
flat_layer = Flatten()(pooled2)
flat_layer = Dropout(rate = 0.25)(flat_layer)
model_output = Dense(10 , activation = 'softmax')(flat_layer)

In [34]:
#檢查model 的STACK
model = Model(inputs = model_input ,
              outputs = model_output)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
__________

In [23]:
model.compile(loss = 'MSE', 
              optimizer = 'adam',
              metrics = ['accuracy'])

train_history = model.fit(x = x_img_train_normalize,
                          y = y_label_train_OneHot , 
                          validation_split = 0.25, 
                          epochs = 12, 
                          batch_size = 128,
                          verbose = 1)

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 143s 4ms/step - loss: 0.0725 - acc: 0.4088 - val_loss: 0.0672 - val_acc: 0.5102
Epoch 2/12
37500/37500 [==============================] - 143s 4ms/step - loss: 0.0597 - acc: 0.5433 - val_loss: 0.0607 - val_acc: 0.5711
Epoch 3/12
37500/37500 [==============================] - 174s 5ms/step - loss: 0.0545 - acc: 0.5913 - val_loss: 0.0554 - val_acc: 0.6227
Epoch 4/12
37500/37500 [==============================] - 199s 5ms/step - loss: 0.0513 - acc: 0.6170 - val_loss: 0.0530 - val_acc: 0.6332
Epoch 5/12
37500/37500 [==============================] - 177s 5ms/step - loss: 0.0488 - acc: 0.6381 - val_loss: 0.0508 - val_acc: 0.6461
Epoch 6/12
37500/37500 [==============================] - 178s 5ms/step - loss: 0.0468 - acc: 0.6568 - val_loss: 0.0494 - val_acc: 0.6593
Epoch 7/12
37500/37500 [==============================] - 188s 5ms/step - loss: 0.0452 - acc: 0.6703 - val_loss: 0.0480 - 

# 訓練模型(loss = 'binary_crossentropy')

In [7]:
model_input_ = Input(shape=(32, 32, 3 , ) , dtype = 'float32' , name = 'main_input')

conv1_ = Conv2D(filters = 32 , 
               kernel_size = (3 , 3) ,
               input_shape = (32 , 32 , 3), 
               activation = 'relu', 
               padding = 'same')(model_input_)

conv1_ = Dropout(rate = 0.25)(conv1_)

pooled1_ = MaxPooling2D(pool_size = (2 , 2))(conv1_)

In [8]:
conv2_ = Conv2D(filters = 64 , 
               kernel_size = (3 , 3) ,
               input_shape = (32 , 32 , 3), 
               activation = 'relu', 
               padding = 'same')(pooled1_)

conv2_ = Dropout(rate = 0.25)(conv2_)

pooled2_ = MaxPooling2D(pool_size = (2 , 2))(conv2_)

In [9]:
flat_layer_ = Flatten()(pooled2_)
flat_layer_ = Dropout(rate = 0.25)(flat_layer_)
model_output_ = Dense(10 , activation = 'softmax')(flat_layer_)

In [11]:
model_ = Model(inputs = model_input_ ,
               outputs = model_output_)


model_.compile(loss = 'binary_crossentropy', 
              optimizer = 'adam',
              metrics = ['accuracy'])

train_history = model_.fit(x = x_img_train_normalize,
                           y = y_label_train_OneHot , 
                           validation_split = 0.25, 
                           epochs = 12, 
                           batch_size = 128,
                           verbose = 1)

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 168s 4ms/step - loss: 0.2439 - acc: 0.9093 - val_loss: 0.2227 - val_acc: 0.9118
Epoch 2/12
37500/37500 [==============================] - 208s 6ms/step - loss: 0.1990 - acc: 0.9225 - val_loss: 0.2019 - val_acc: 0.9195
Epoch 3/12
37500/37500 [==============================] - 223s 6ms/step - loss: 0.1834 - acc: 0.9285 - val_loss: 0.1896 - val_acc: 0.9244
Epoch 4/12
37500/37500 [==============================] - 220s 6ms/step - loss: 0.1724 - acc: 0.9326 - val_loss: 0.1849 - val_acc: 0.9282
Epoch 5/12
37500/37500 [==============================] - 222s 6ms/step - loss: 0.1645 - acc: 0.9360 - val_loss: 0.1725 - val_acc: 0.9327
Epoch 6/12
37500/37500 [==============================] - 174s 5ms/step - loss: 0.1577 - acc: 0.9389 - val_loss: 0.1677 - val_acc: 0.9345
Epoch 7/12
37500/37500 [==============================] - 172s 5ms/step - loss: 0.1530 - acc: 0.9403 - val_loss: 0.1669 - 